In [1]:
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0)",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_0 like Mac OS X)"
]

In [3]:
import random
def get_headers():
    return {
        "User-Agent": random.choice(user_agents),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Referer": "https://www.google.com/"
    }

In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import random

def get_ilan_links(ana_sayfa_url, ilan_sayfa_sayisi):
    
    ilan_linkleri = set() 
    
    for i in range(1, ilan_sayfa_sayisi + 1):
        sayfa_url = f"{ana_sayfa_url}?page={i}"
        print(f"[+] Sayfa {i}: {sayfa_url}")
        
        max_retries = 3
        for retry in range(max_retries):
            try:
                # Daha doğal bekleme süreleri ve rastgele User-Agent
                time.sleep(random.uniform(10, 15))
                
                response = requests.get(sayfa_url, headers=get_headers(), timeout=20)
                
                # Debug için HTML kaydet
                with open(f"debug_page_{i}.html", "w", encoding="utf-8") as f:
                    f.write(response.text)
                
                if response.status_code != 200:
                    print(f"HTTP Hatası: {response.status_code}")
                    if retry < max_retries - 1:
                        print(f"Yeniden deneniyor ({retry+1}/{max_retries})...")
                        time.sleep(30)  # Hata durumunda daha uzun bekleme
                        continue
                    else:
                        break
                
                soup = BeautifulSoup(response.text, 'lxml')
                
                # Daha geniş seçici kümesi kullan
                ilan_satirlari = soup.find_all('tr', {'class': 'listing-list-item'})
                
                if not ilan_satirlari or len(ilan_satirlari) == 0:
                    print("İlan satırları bulunamadı, alternatif seçiciler deneniyor...")
                    # Alternatif seçici dene
                    ilan_satirlari = soup.select('.listing-list-item, .car-listing, [id^="listing"]')
                
                print(f"Sayfa {i}: {len(ilan_satirlari)} ilan satırı bulundu.")
                
                sayfa_ilan_sayisi = 0
                for satir in ilan_satirlari:
                    try:
                        # Birden fazla seçici dene
                        link_element = satir.find('a', {'class': 'link-overlay'})
                        if not link_element:
                            link_elements = satir.find_all('a', href=True)
                            for el in link_elements:
                                href = el.get('href', '')
                                if '/ilan/' in href:
                                    link_element = el
                                    break
                        
                        if link_element and link_element.get('href'):
                            link = link_element.get('href')
                            
                            if link and '/ilan/' in link:
                                tam_link = f"https://www.arabam.com{link}" if not link.startswith('http') else link
                                ilan_linkleri.add(tam_link)
                                sayfa_ilan_sayisi += 1
                    except Exception as e:
                        print(f"İlan ayrıştırma hatası: {str(e)}")
                        continue
                
                print(f"Sayfa {i}'de {sayfa_ilan_sayisi} yeni ilan bulundu. Toplam: {len(ilan_linkleri)}")
                
                # Sayfada hiç ilan bulunamadıysa ve bu ilk deneme değilse
                if sayfa_ilan_sayisi == 0 and retry < max_retries - 1:
                    print(f"Bu sayfada ilan bulunamadı, yeniden deneniyor ({retry+1}/{max_retries})...")
                    time.sleep(random.uniform(30, 45))  # Daha uzun bekle ve tekrar dene
                    continue
                else:
                    # Başarılı çekme veya tüm denemelerin tükenmesi
                    break
                
            except Exception as e:
                print(f"Hata: {str(e)}")
                if retry < max_retries - 1:
                    print(f"Yeniden deneniyor ({retry+1}/{max_retries})...")
                    time.sleep(30)  # Hata durumunda daha uzun bekleme
                else:
                    print("Maksimum deneme sayısına ulaşıldı, sonraki sayfaya geçiliyor.")
        
        # Sayfalar arası daha uzun bekleme
        wait_time = random.uniform(15, 25)
        print(f"Sonraki sayfaya geçmeden önce {wait_time:.1f} saniye bekleniyor...")
        time.sleep(wait_time)
    
    print(f"Toplam {len(ilan_linkleri)} ilan linki bulundu.")
    return list(ilan_linkleri)

In [5]:
# Buraya hangi arabaların ilanlarını toplamak istediğinizin URL'sini yazın
ana_sayfa_url = "https://www.arabam.com/ikinci-el/otomobil/tesla-model-y"
#İkinci parametre olarak arabanın kaç sayfa ilanı olduğunu yazın
ilan_linkleri = get_ilan_links(ana_sayfa_url,7)

[+] Sayfa 1: https://www.arabam.com/ikinci-el/otomobil/tesla-model-y?page=1
Sayfa 1: 20 ilan satırı bulundu.
Sayfa 1'de 20 yeni ilan bulundu. Toplam: 20
Sonraki sayfaya geçmeden önce 31.8 saniye bekleniyor...
[+] Sayfa 2: https://www.arabam.com/ikinci-el/otomobil/tesla-model-y?page=2
Sayfa 2: 20 ilan satırı bulundu.
Sayfa 2'de 20 yeni ilan bulundu. Toplam: 40
Sonraki sayfaya geçmeden önce 34.0 saniye bekleniyor...
[+] Sayfa 3: https://www.arabam.com/ikinci-el/otomobil/tesla-model-y?page=3
Sayfa 3: 20 ilan satırı bulundu.
Sayfa 3'de 20 yeni ilan bulundu. Toplam: 60
Sonraki sayfaya geçmeden önce 38.6 saniye bekleniyor...
[+] Sayfa 4: https://www.arabam.com/ikinci-el/otomobil/tesla-model-y?page=4
Sayfa 4: 20 ilan satırı bulundu.
Sayfa 4'de 20 yeni ilan bulundu. Toplam: 80
Sonraki sayfaya geçmeden önce 39.0 saniye bekleniyor...
[+] Sayfa 5: https://www.arabam.com/ikinci-el/otomobil/tesla-model-y?page=5
Sayfa 5: 20 ilan satırı bulundu.
Sayfa 5'de 20 yeni ilan bulundu. Toplam: 100
Sonraki sa

In [6]:
print(ilan_linkleri)

['https://www.arabam.com/ilan/galeriden-satilik-tesla-model-y-long-range-awd/galeriden-tesla-model-y-long-range-awd-2023-model-eskisehir/28961640', 'https://www.arabam.com/ilan/galeriden-satilik-tesla-model-y-long-range-awd/sclass-2023-tesla-model-y-long-range-514-hp-4wd-fsd-hatasiz/29229233', 'https://www.arabam.com/ilan/galeriden-satilik-tesla-model-y-performance/uniq-auto-2023-model-y-performance-o-pilot-4k-istma-bysz/29047324', 'https://www.arabam.com/ilan/sahibinden-satilik-tesla-model-y-standart-range/sahibinden-tesla-model-y-standart-range-2025-model/28939710', 'https://www.arabam.com/ilan/galeriden-satilik-tesla-model-y-performance/uniq-auto-2023-tesla-model-y-performance-cam-tvn-otonom-360k/29047312', 'https://www.arabam.com/ilan/galeriden-satilik-tesla-model-y-performance/baran-motors-2023-tesla-modely-performance-full-self-driving-ppf/27813499', 'https://www.arabam.com/ilan/sahibinden-satilik-tesla-model-y-long-range-awd/sahibinden-tesla-model-y-long-range-awd-2023-model/284

In [ ]:
def get_ilan_detaylari(url):
    max_retries = 3
    
    for retry in range(max_retries):
        try:
            # Doğal bekleme süresi ekle
            time.sleep(random.uniform(5, 10))
            
            response = requests.get(url, headers=get_headers(), timeout=20)
            
            
            if response.status_code != 200:
                print(f"HTTP Hatası: {response.status_code}")
                if retry < max_retries - 1:
                    print(f"Yeniden deneniyor ({retry+1}/{max_retries})...")
                    time.sleep(30)  # Hata durumunda daha uzun bekleme
                    continue
                else:
                    return None
            
            soup = BeautifulSoup(response.text, 'lxml')
            
            detaylar = {}
            
            # Fiyat bilgisini çek - birden fazla seçici dene
            try:
                fiyat = soup.find('div', {'data-testid': 'desktop-information-price'}).text.strip()
                detaylar['Fiyat'] = fiyat
            except:
                detaylar['Fiyat'] = 'Belirtilmemiş'
            
            property_items = soup.find_all('div', {'class': 'property-item'})
            for item in property_items:
                try:
                    key = item.find('div', {'class': 'property-key'}).text.strip()
                    value = item.find('div', {'class': 'property-value'}).text.strip()
                    detaylar[key] = value
                except:
                     continue
            
            
            # Parça durumlarını çek
            parca_isimleri = {
                "Sol Arka Kapı":"Belirtilmemiş",
                "Sol Ön Kapı":"Belirtilmemiş",
                "Ön Tampon":"Belirtilmemiş",
                "Sol Arka Çamurluk":"Belirtilmemiş",
                "Sol Ön Çamurluk":"Belirtilmemiş",
                "Sağ Ön Çamurluk":"Belirtilmemiş",
                "Sağ Arka Kapı":"Belirtilmemiş",
                "Sağ Ön Kapı":"Belirtilmemiş",
                "Sağ Arka Çamurluk":"Belirtilmemiş",
                "Motor Kaputu":"Belirtilmemiş",
                "Arka Kaput":"Belirtilmemiş",
                "Tavan":"Belirtilmemiş",
                "Arka Tampon":"Belirtilmemiş"
            }
            parca_durumlari = parca_isimleri.copy()

            paths = soup.find_all('path', {'uib-tooltip': True})

            for i, path in enumerate(paths):
                try:
                    if i >= len(parca_isimleri):
                        continue

                    parca_adi = list(parca_isimleri.keys())[i]
                    durum = path.get('uib-tooltip', 'Belirtilmemiş')

                    parca_durumlari[parca_adi] = durum
                except Exception as e:
                    print(f"Değişen durumunda hata: {str(e)}")
            
                detaylar.update(parca_durumlari)
            
            return detaylar
            
        except Exception as e:
            print(f"İlan detay çekme hatası: {str(e)}")
            if retry < max_retries - 1:
                print(f"Yeniden deneniyor ({retry+1}/{max_retries})...")
                time.sleep(random.uniform(20, 30))
            else:
                print(f"Maksimum deneme sayısına ulaşıldı, ilan atlanıyor: {url}")
                return None

In [11]:
import pandas as pd
def process_all_ilans(ilan_linkleri):
    all_details = []
    
    for idx, link in enumerate(ilan_linkleri, start=1):
        print(f"[{idx}/{len(ilan_linkleri)}] İlan {link}")
        detaylar = get_ilan_detaylari(link)
        if detaylar:
            detaylar['ID'] = idx
            all_details.append(detaylar)
            
        time.sleep(random.uniform(15,25))
        
    df = pd.DataFrame(all_details)
    
    df.to_csv("ilan_detaylari.csv", index=False, encoding='utf-8-sig')
    print("Başarılı")

In [12]:
process_all_ilans(ilan_linkleri)

[1/132] İlan https://www.arabam.com/ilan/galeriden-satilik-tesla-model-y-long-range-awd/galeriden-tesla-model-y-long-range-awd-2023-model-eskisehir/28961640
[2/132] İlan https://www.arabam.com/ilan/galeriden-satilik-tesla-model-y-long-range-awd/sclass-2023-tesla-model-y-long-range-514-hp-4wd-fsd-hatasiz/29229233
[3/132] İlan https://www.arabam.com/ilan/galeriden-satilik-tesla-model-y-performance/uniq-auto-2023-model-y-performance-o-pilot-4k-istma-bysz/29047324
[4/132] İlan https://www.arabam.com/ilan/sahibinden-satilik-tesla-model-y-standart-range/sahibinden-tesla-model-y-standart-range-2025-model/28939710
[5/132] İlan https://www.arabam.com/ilan/galeriden-satilik-tesla-model-y-performance/uniq-auto-2023-tesla-model-y-performance-cam-tvn-otonom-360k/29047312
[6/132] İlan https://www.arabam.com/ilan/galeriden-satilik-tesla-model-y-performance/baran-motors-2023-tesla-modely-performance-full-self-driving-ppf/27813499
[7/132] İlan https://www.arabam.com/ilan/sahibinden-satilik-tesla-model-